In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
from dataset import NBA_Dataset
from tqdm import tqdm
import numpy as np

def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [45]:
window = 7
workers = 0
EPOCH = 100
BATCH_SIZE = 200
LR = 0.001

In [21]:
train_dataset = NBA_Dataset("train_data", "column_mean_and_std.csv", window=window)
val_dataset = NBA_Dataset("val_data", "column_mean_and_std.csv", window=window)
# nba_dataset = NBA_Dataset("train_data", "column_mean_and_std.csv", window=window)


100%|██████████| 30/30 [00:04<00:00,  6.55it/s]


In [48]:
class Net(nn.Module):
    def __init__(self,input_size):
        super(Net, self).__init__()
        drop = 0.5
        self.inp = nn.Sequential(
            nn.Linear(input_size,128),
            nn.Dropout(drop),
            nn.BatchNorm1d(128),
            nn.ReLU(),
        )
        self.L1 = nn.Sequential(
            nn.Linear(128,64),
            nn.Dropout(drop),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,128),
            nn.Dropout(drop),
            nn.BatchNorm1d(128),
        )
        self.out = nn.Sequential(
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x1 = self.inp(x)
        x = self.L1(x1)
        x = self.out(x+x1)
        return x

In [ ]:
same_seeds(1)
# train_dataset, val_dataset = random_split(nba_dataset, [0.8, 0.2])

print("資料集大小:", len(train_dataset), len(val_dataset))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=workers)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=workers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(train_dataset[0][0].shape[0]).to(device)
# model.load_state_dict(torch.load("modelacc.pth",weights_only=True))
print("輸入大小:", train_dataset[0][0].shape[0])
print("模型大小:", sum(p.numel() for p in model.parameters() if p.requires_grad))

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LR,weight_decay=8e-3)
# optimizer = torch.optim.SGD(model.parameters(), lr=LR,weight_decay=5e-4,momentum=0.9)

maxacc = 0
minloss = 1000
for epoch in range(EPOCH):
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0

    model.train()
    for data, label in train_loader:
        data, label = data.to(device), label.to(device)
        output = model(data)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)
        optimizer.step()
        train_loss += loss.item()
        train_acc += ((output > 0.5).eq(label > 0.5)).sum().item()


    model.eval()
    with torch.no_grad():
        for data, label in val_loader:
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = criterion(output, label)
            val_loss += loss.item()
            val_acc += ((output > 0.5).eq(label > 0.5)).sum().item()

    print(f"Epoch: {epoch}, Train Loss: {train_loss / len(train_loader):.6f}, Train Acc: {train_acc / len(train_dataset):.6f}, Val Loss: {val_loss / len(val_loader):.6f}, Val Acc: {val_acc / len(val_dataset):.6f}")
    if val_acc / len(val_dataset) > maxacc:
        maxacc = val_acc / len(val_dataset)
        print("save model with acc:", maxacc)
        torch.save(model.state_dict(), "modelacc.pth")
    if val_loss / len(val_loader) < minloss:
        minloss = val_loss / len(val_loader)
        print("save model with loss:", minloss)
        torch.save(model.state_dict(), "modelloss.pth")


資料集大小: 9332 3340
輸入大小: 465
模型大小: 38721
Epoch: 0, Train Loss: 0.647366, Train Acc: 0.622696, Val Loss: 0.663352, Val Acc: 0.596407
save model with acc: 0.5964071856287425
save model with loss: 0.6633522510528564
Epoch: 1, Train Loss: 0.648687, Train Acc: 0.627733, Val Loss: 0.663425, Val Acc: 0.594910
Epoch: 2, Train Loss: 0.645479, Train Acc: 0.623125, Val Loss: 0.663253, Val Acc: 0.600599
save model with acc: 0.6005988023952096
save model with loss: 0.6632525465067696
Epoch: 3, Train Loss: 0.645966, Train Acc: 0.623768, Val Loss: 0.663461, Val Acc: 0.597305
Epoch: 4, Train Loss: 0.643606, Train Acc: 0.629018, Val Loss: 0.663477, Val Acc: 0.596407
Epoch: 5, Train Loss: 0.648845, Train Acc: 0.621517, Val Loss: 0.663461, Val Acc: 0.596407
Epoch: 6, Train Loss: 0.646263, Train Acc: 0.625161, Val Loss: 0.663439, Val Acc: 0.596407
Epoch: 7, Train Loss: 0.646718, Train Acc: 0.623875, Val Loss: 0.663673, Val Acc: 0.592814
Epoch: 8, Train Loss: 0.644141, Train Acc: 0.626982, Val Loss: 0.663363

KeyboardInterrupt: 